
## OPTION PRICING




In [1]:
import numpy as np

Below is the code for a general function to simulate the underlying stock price for some inputs: initial stock price (`S_ini`), time-horizon ($T$), upward ($u$) and downward ($d$) movements, and number of steps (N).

In [2]:
def binomial_tree(S_ini, T, u, d, N):
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return S

Note that we store everything in a variable, $S$, the one returned by the function. This variable will contain an array with the values of the stock price at each point in time in a lower triangular matrix.

Let's check it by replicating the same tree for $N=2$ that you have in the Lesson 1 slides:

In [3]:
Stock = binomial_tree(100, 1, 1.2, 0.8, 2)
Stock

array([[100.,   0.,   0.],
       [ 80., 120.,   0.],
       [ 64.,  96., 144.]])

## 2. Extending the Tree with Call Option Payoffs

Next, let's extend the previous function by adding another variable that computes the payoffs associated with a Call Option of certain characteristics. Note that we are focusing on a European Call Option with strike price $K=90$, and therefore the payoff is only computed at maturity:

In [4]:
def binomial_tree_call(S_ini, K, T, u, d, N):
    C = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        C[N, i] = max(S_ini * (u ** (i)) * (d ** (N - i)) - K, 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return S, C

In [5]:
Stock, Call = binomial_tree_call(100, 90, 10, 1.2, 0.8, 10)
print("Underlying Price Evolution:\n", Stock)
print("Call Option Payoff:\n", Call)

Underlying Price Evolution:
 [[100.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 80.         120.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 64.          96.         144.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 51.2         76.8        115.2        172.8          0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 40.96        61.44        92.16       138.24       207.36
    0.           0.           0.           0.           0.
    0.        ]
 [ 32.768       49.152       73.728      110.592      165.888
  248.832        0.           0.           0.           0.
    0.        ]
 [ 26.2144      39.3216      58.9824      88.4736     132.7104
  199.0656     298.5984       0.           0.           0.
    0.        ]
 [ 20.97152     31.45728

It is easy to see that the variable $C$ output by the function will return Call Option payoff at maturity. We can verify this by replicating the simple N=1 tree with Call option payoff from the Lesson 1 slides:

## 3. Introducing Risk-Neutral Probabilities and backward induction of Call Option Value

For the final part of this notebook, let's work with the risk-neutral probabilities. Once we have the probabilities, we can, by backward induction, calculate the value of the Call Option (given its future payoffs and the associated probabilities) at each node.

Importantly, once we know the risk-neutral probabilities, the value of the Call Option at each node will depend on the expected payoff in the two potential future scenarios (up or down movements), discounted at risk-free. That is:

$C_{t}= e^{-rdt}[p c_{t+1}^u + (1-p) c_{t+1}^d]$

where $dt$ is the discounted period from one node to the next (**time-step**), and $c_{t+1}^u$ and $c_{t+1}^d$ are the **values of the Call option in the next period**. We will therefore have to start from the last period (maturity) and work backwards, hence the term backward induction.

Note that we use $dt$ here because we are assuming there are a **bunch of periods (steps) in the tree from the initial date until maturity of the option**. Under the 1-step case, we can calculate risk-neutral probabilities as we did in the videos, because $dt=T/N = 1/1 = 1 = T$:

$p=\frac{e^{rT}-d}{u-d}$

Once we consider a different $dt$, we just need to modify $p$ accordingly:

$p=\frac{e^{rdt}-d}{u-d}$

Now, let's write a final function that recognizes all these issues.

In [6]:
def binomial_call_full(S_ini, K, T, r, u, d, N):
    dt = T / N  # Define time step
    p = (np.exp(r * dt) - d) / (u - d)  # Risk neutral probabilities (probs)
    C = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        C[N, i] = max(S_ini * (u ** (i)) * (d ** (N - i)) - K, 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            C[j, i] = np.exp(-r * dt) * (p * C[j + 1, i + 1] + (1 - p) * C[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return C[0, 0], C, S

Notice that since we are doing backward induction, the first value of the Call Option Payoff matrix (the last we calculate) is the price of the Call Option today.

Let's replicate with the values from the example in the slides to check it:

In [7]:
call_price, C, S = binomial_call_full(100, 90, 10, 0, 1.2, 0.8, 10)
print("Underlying Price Evolution:\n", S)
print("Call Option Payoff:\n", C)
print("Call Option Price at t=0: ", "{:.2f}".format(call_price))

Underlying Price Evolution:
 [[100.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 80.         120.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 64.          96.         144.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 51.2         76.8        115.2        172.8          0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 40.96        61.44        92.16       138.24       207.36
    0.           0.           0.           0.           0.
    0.        ]
 [ 32.768       49.152       73.728      110.592      165.888
  248.832        0.           0.           0.           0.
    0.        ]
 [ 26.2144      39.3216      58.9824      88.4736     132.7104
  199.0656     298.5984       0.           0.           0.
    0.        ]
 [ 20.97152     31.45728